In [ ]:
# Import packages
import TS_Custom
from TS_Custom import *


#initialize globalvariables
buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
    
def getStrategyResult(symbol):
    #for symbol in symbol_list :
    try:
        #print (datetime.now().strftime('%H:%M'))            
        # get price history from tradestation
        mins = 2
        
        #bars are set to 600 so that ATR for UTBot Buy is accurate     
        barsBack=int(barssince7am(mins))
        if (barsBack <= 15):
            barsBack = 15
        
        df = getPriceHistory_TS(symbol,str(barsBack), str(mins))
        df = getHA(df)
       
        # get options data from td ameritrade
        content = getOptionChainDatafromTD(symbol)
        # retrieve call symbol
        call_optionSymbol = getCallOptionSymbol(content)

        # retrieve put symbol
        put_optionSymbol = getPutOptionSymbol(content) 
        
        df = Supertrend(df, 10, 1)
        
        atr = df['ATR']
        ema9 = df['ema9']
        ema21 = df['ema21']
        
        #retrieve keys     
        last_key = atr.keys()[-1]
        previous_key = atr.keys()[-2]
        previous_key_2 = atr.keys()[-3]
        previous_key_3 = atr.keys()[-4]
        
        # Strategy Code
        #variables
        datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        strategy = 'BUY_STRATS'

        # Buy Stock  
        if (buylong == True):
            
            if (
            ((df['High'][previous_key_2] > df['High'][previous_key_3]) and \
            (df['Low'][previous_key_2] < df['Low'][previous_key_3]) and \
            (df['High'][previous_key] < df['High'][previous_key_2]) and \
            (df['Low'][previous_key] > df['Low'][previous_key_2]))) :
                
                if (not getSymbolStockPositionsfromDB(symbol)) :                        

                    # get quote from tradestation
                    quote = getQuote(symbol)
                    #midPrice = round((quote['Bid'] + quote['Ask'])/2, 2)
                    bidPrice = round((quote['Bid']), 2)
                    askPrice = round((quote['Ask']), 2)
                    midPrice = round((askPrice + bidPrice)/2,2)                    
                    limitprice = float(round(askPrice+0.01,2))

                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)   

                    if ((equity_bidaskpercent <= 0.4) and\
                        (askPrice > df['High'][previous_key])):
                        
                        quantity=int(3000/(limitprice))

                        print("ALL CONDITIONS FOR BUY STOCK MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (bidPrice-limitprice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = round((quantity * bidPrice),2)
                        triggerprice = round(quantity * 0.40 * atr[last_key],2)
                        previous_openpl = round((quantity * (bidPrice-limitprice)), 2)
                        profit_target = round(limitprice * 1.03,2)
                        stop_loss = round(limitprice * 0.995,2)

                        #buy Equity in TS
                        buyEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)

                        # add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'BUY_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Long', quantity, limitprice,totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df[['SYMBOL','TimeStamp','Close','Open','High','Low']].tail(5))
                        #df.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'BUY_STOCK', limitprice, quantity, '', bidPrice, askPrice,df)




        # Sell Short Stock   
        if (buyshort == True):
            
            if (
            ((df['High'][previous_key_2] > df['High'][previous_key_3]) and \
            (df['Low'][previous_key_2] < df['Low'][previous_key_3]) and \
            (df['High'][previous_key] < df['High'][previous_key_2]) and \
            (df['Low'][previous_key] > df['Low'][previous_key_2]))) :
                
                if (not getSymbolStockPositionsfromDB(symbol)) :

                    # get quote from tradestation
                    quote = getQuote(symbol)
                    askPrice = round((quote['Ask']), 2)
                    bidPrice = round((quote['Bid']), 2)
                    midPrice = round((askPrice + bidPrice)/2,2)
                    limitprice = float(round(bidPrice-0.01,2))

                    equity_bidaskpercent = getEquity_BidAskPercent(symbol,bidPrice,askPrice)
                    #print("equity_bidaskpercent:",equity_bidaskpercent, 'for', symbol)

                    if ((equity_bidaskpercent <= 0.4) and \
                        (bidPrice < df['Low'][previous_key])):

                        quantity=int(3000/(limitprice))

                        print("ALL CONDITIONS FOR SELLSHORT MET FOR ", symbol)

                        totalCost = round((limitprice*quantity),2)
                        openpl = round((quantity * (limitprice-askPrice)), 2)
                        openpl_percent = round(((openpl/totalCost)*100), 2)
                        mktvalue = round((quantity * bidPrice),2)

                        #triggerprice = round(quantity * atr[last_key],2)
                        triggerprice = round(quantity * 0.40 * atr[last_key],2)
                        previous_openpl = round((quantity * (limitprice-askPrice)), 2)
                        #profit_target = limitprice * 0.935
                        profit_target = round(limitprice * 0.97,2)
                        #profit_target = limitprice - float(triggerprice/quantity)
                        stop_loss = round(limitprice * 1.005,2)

                        #sellshort Equity in TS
                        sellshortEQUITY(symbol, quantity, limitprice, tradelog_filename)

                        winsound.Beep(4000, 500)

                        #add to DB
                        addOrderOutputtoDB(datetime_now, str(symbol), 'SELLSHORT_STOCK', limitprice, quantity)
                        addPositionOutputtoDB(datetime_now, str(symbol), 'Short', quantity, limitprice, totalCost, openpl,openpl_percent,bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)

                        print(df[['SYMBOL','TimeStamp','Close','Open','High','Low']].tail(5))
                        #df.to_csv(buystochasticresults_filename, mode = 'a', index=False)

                        #add log to DB
                        addOrderLogOutputtoDB(datetime_now, symbol, strategy,'SELLSHORT_STOCK', limitprice, quantity, '', bidPrice, askPrice,df)


        
        
        '''
        # Buy Options
        if (buycall == True):

            # Call Option
            if ((df['ema9_angle'][previous_key] >= 25) and \
                (df['ema9_angle'][previous_key] > df['ema9_angle'][previous_key_2]) and \
                (df['linear_reg_slope'][previous_key] > df['linear_reg_slope'][previous_key_2]) and \
                (df['HA_Close'][previous_key] > df['HA_Open'][previous_key]) and \
                #(df['HA_Close'][last_key] > df['HA_Open'][last_key]) and \
                (rsi[previous_key] <= 70) and \
                (ema9[previous_key] > ema21[previous_key]) and \
                (ema21[previous_key] > ema50[previous_key])) :

                if (((df['linear_reg_slope'][previous_key] > 0) and \
                    (df['linear_reg_slope'][previous_key_2] < 0)) or \
                    ((df['linear_reg_slope'][previous_key] > 0) and \
                     (df['linear_reg_slope'][previous_key_2] > 0) and \
                     (df['linear_reg_slope'][previous_key_3] < 0))) : 


                    #if (not getSymbolCallPositions(acct,symbol)) and (not getSymbolCallOrders_Received(acct,symbol)) :
                    if (not getSymbolCallPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "callExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "callExpDateMap")
                            bidPrice = getBidPrice(content, "callExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(15000/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY CALL OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Call Options in TS
                            buyCALL(symbol, quantity, df, call_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(call_optionSymbol), 'BUY_CALL', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(call_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(call_optionSymbol), strategy,'BUY_CALL', limitprice, quantity, '', bidPrice, askPrice,df)



        # Put Option

        if (buyput == True):

            if ((df['ema9_angle'][previous_key] < 0) and \
                (df['linear_reg_slope'][previous_key] < df['linear_reg_slope'][previous_key_2]) and \
                (df['ema9_angle'][last_key] < 0) and \
                (df['HA_Close'][previous_key] < df['HA_Open'][previous_key]) and \
                #(df['HA_Close'][last_key] < df['HA_Open'][last_key]) and \
                (rsi[previous_key] >=35 ) and \
                (ema9[previous_key] < ema21[previous_key]) and \
                (ema21[previous_key] < ema50[previous_key])) :

                if (((df['linear_reg_slope'][previous_key] < 0) and \
                    (df['linear_reg_slope'][previous_key_2] > 0)) or \
                    ((df['linear_reg_slope'][previous_key] < 0) and \
                     (df['linear_reg_slope'][previous_key_2] < 0) and \
                     (df['linear_reg_slope'][previous_key_3] > 0))) : 

                    #if (not getSymbolPutPositions(acct,symbol)) and (not getSymbolPutOrders_Received(acct,symbol)) :
                    if (not getSymbolPutPositionsfromDB(symbol)) :

                        bidaskpercent = getBidAskPercent(content, "putExpDateMap")
                        print("bidaskpercent:",bidaskpercent,'% for', symbol)
                        if bidaskpercent <= 5 :
                            askPrice = getAskPrice(content, "putExpDateMap")
                            bidPrice = getBidPrice(content, "putExpDateMap")
                            limitprice = askPrice
                            print('limit',limitprice)

                            if (symbol == "TSLA") or (symbol == "GOOG") :
                                quantity = 2
                            else :
                                quantity=int(15000/(limitprice*100))

                            print("ALL CONDITIONS FOR BUY PUT OPTION MET FOR ", symbol)

                            totalCost = round((limitprice*quantity*100),2)
                            openpl = round((quantity * 100* (bidPrice-limitprice)), 2)
                            openpl_percent = round(((openpl/totalCost)*100), 2)
                            mktvalue = mktvalue = round((quantity * 100 * bidPrice),2)
                            #triggerprice = round(limitprice + atr[last_key],2)
                            triggerprice = round(quantity * atr[last_key],2)
                            previous_openpl = round((quantity * 100 * (bidPrice-limitprice)), 2)
                            profit_target = limitprice * 1.0065
                            stop_loss = limitprice * 0.995

                            winsound.Beep(4000, 500)

                            #Buy Put Options in TS
                            buyPUT(symbol, quantity, df, put_optionSymbol, tradelog_filename, content)


                            # add to DB
                            addOrderOutputtoDB(datetime_now, str(put_optionSymbol), 'BUY_PUT', limitprice, quantity)
                            addPositionOutputtoDB(datetime_now, str(put_optionSymbol), 'Long', quantity, limitprice,totalCost, openpl, openpl_percent, bidPrice,askPrice,mktvalue, triggerprice,previous_openpl,strategy,profit_target,stop_loss)


                            #add log to DB
                            addOrderLogOutputtoDB(datetime_now, str(put_optionSymbol), strategy,'BUY_PUT', limitprice, quantity, '', bidPrice, askPrice,df)

            '''
            #print("******************")

    except Exception as e:
        print(symbol)
        traceback.print_exc()
        print(e)
        
while True:        
        
    now = datetime.now()
    today800=now.replace(hour=8,minute=0,second=0,microsecond=0)
    today1500=now.replace(hour=15,minute=0,second=0,microsecond=0)
    #if (True) :
    if (now > today800 and now < today1500 ) :
        #import Multithreading file for this function below
        #symbol_list = getMomentumStocks()
        #print(symbol_list)
        #getStrategyResult(symbol_list)

        #get stocks list from multithread YF data
        #symbol_list = getStocks()
        #print('SYMBOL LIST', symbol_list)        

        buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()

        if (buystrategy == True) :   
            
            #symbol_list1 = ['AAPL'] 
            
                      
            symbol_list1 = pd.read_csv('tickers1.csv', header=None)[0].tolist()
            print(symbol_list1)
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list1)) as executor:
                executor.map(getStrategyResult,symbol_list1)

            # Total time calculated
            print(now, ' - symbol_list1 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)


            '''
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list2 = pd.read_csv('tickers2.csv', header=None)[0].tolist()
            print(symbol_list2)
            
            
            # Time starts from here
            start = time.time()


            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list2)) as executor:            
                executor.map(getStrategyResult,symbol_list2)

            # Total time calculated
            print(now, ' - symbol_list2 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)
            
            
            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()
            symbol_list3 = pd.read_csv('tickers3.csv', header=None)[0].tolist()
            print(symbol_list3)
            
            # Time starts from here
            start = time.time()        

            with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list3)) as executor:
                #print(threading.currentThread().ident)
                executor.map(getStrategyResult,symbol_list3)

            # Total time calculated
            print(now, ' - symbol_list3 took ', time.time()-start, 'seconds.')

            print("**************************************************************")
            time.sleep(30)
            
                       

            
            buystrategy,sellstrategy,buylong,buyshort,buycall,buyput = get_Trade_Signals_fromDB()  
            #symbol_list0=getMomentumStocks_TS_Scanner().head(10)['Symbol'].tolist()
            momentum_df=getVolumeMomentumStocks_TS_Scanner()
            if (not momentum_df.empty):
                symbol_list0=momentum_df.head(10)['Symbol'].tolist()
                print("Momentum Scanner Stocks:")
                print(symbol_list0)
                # Time starts from here
                start = time.time()
                if(len(symbol_list0)>0):
                    with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list0)) as executor:
                        executor.map(getStrategyResult,symbol_list0)

                # Total time calculated
                print(now, ' - symbol_list0 took ', time.time()-start, 'seconds.')

                print("**************************************************************")
                time.sleep(30)

            '''
        
    
   




['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:27:03.022859  - symbol_list1 took  1.41015625 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:27:34.502673  - symbol_list1 took  1.2397654056549072 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Expecting value: line 1 column 1 (char 0)
list indices must be integers or slices, not str
TSLA
list indices must be integers or slices, not str
2023-01-05 09:28:05.841322  - symbol_list1 took  7.852588176727295 seconds.
**************************************************************


Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_12188\1183571934.py", line 32, in getStrategyResult
    df = Supertrend(df, 10, 1)
  File "C:\Test_AlphaOne\TS_Custom.py", line 3683, in Supertrend
    high = df['HA_High']
TypeError: list indices must be integers or slices, not str


['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:28:43.738992  - symbol_list1 took  1.0549719333648682 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:29:14.869337  - symbol_list1 took  1.1083567142486572 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:29:46.061789  - symbol_list1 took  1.5369353294372559 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:30:17.634406  - symbol_list1 took  1.0437533855438232 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'N

2023-01-05 09:45:25.414094  - symbol_list1 took  1.0469350814819336 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:45:56.514138  - symbol_list1 took  1.239633560180664 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:46:27.805362  - symbol_list1 took  0.9869451522827148 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
Expecting value: line 1 column 1 (char 0)
list indices must be integers or slices, not str
NFLX
list indices must be integers or slices, not str


Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_12188\1183571934.py", line 32, in getStrategyResult
    df = Supertrend(df, 10, 1)
  File "C:\Test_AlphaOne\TS_Custom.py", line 3683, in Supertrend
    high = df['HA_High']
TypeError: list indices must be integers or slices, not str


Expecting value: line 1 column 1 (char 0)
list indices must be integers or slices, not str
AMZN
list indices must be integers or slices, not str
2023-01-05 09:46:58.835997  - symbol_list1 took  17.007585763931274 seconds.
**************************************************************


Traceback (most recent call last):
  File "C:\Users\VivekP\AppData\Local\Temp\ipykernel_12188\1183571934.py", line 32, in getStrategyResult
    df = Supertrend(df, 10, 1)
  File "C:\Test_AlphaOne\TS_Custom.py", line 3683, in Supertrend
    high = df['HA_High']
TypeError: list indices must be integers or slices, not str


['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:47:45.891802  - symbol_list1 took  1.3482575416564941 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:48:17.282581  - symbol_list1 took  1.0870990753173828 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:48:48.435266  - symbol_list1 took  1.076902151107788 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NFLX', 'AAPL', 'MSFT', 'AMZN']
2023-01-05 09:49:19.554492  - symbol_list1 took  1.3831627368927002 seconds.
**************************************************************
['LABU', 'SPY', 'COIN', 'SHOP', 'TSLA', 'NVDA', 'AMD', 'CCL', 'NF